# Импорты

In [2]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 13.1 MB/s eta 0:00:00


In [3]:
import pymorphy3

In [4]:
import pandas as pd
from typing import Union
import re


Добавить удаление дубликатов!!

# Склейка датасета

In [2]:
data = pd.read_csv('/content/склеить - Лист1 (2).csv')
data.head()

,column 1,column 2,column 3
0,"периодически весьма благозвучное: ""чувак"", """,чувиха,""" и ""жопа"" (последнее уже от начальника,"
1,и при этом какая прекрасная простая лучезарная,чувиха,! ¶ Что они предлагают со второго раза отбирать
2,кастрированного барана или верблюда. А вот,чувиха,"- это его, барана, соломенная подстилка. ¶ Не"
3,", религиозные темы, всякую готику и мрак.",Чувихи,"– японских карпиков кой , портретики"
4,"Ведь в голове у всех Виллы и мерседесы,красивые",чувихи,"и апельсиновой рай.Да ну какая Россия,мы давно"


In [3]:
columns_to_merge = ['column 1', 'column 2', 'column 3']

In [4]:
data['united'] = data[columns_to_merge].apply(
    lambda row: ' '.join(row.values.astype(str)),axis=1
)

In [5]:
output_path = 'результат_склейки1.xlsx'
data.to_excel(output_path, index=False)

In [6]:
data = pd.read_csv('/content/склеить НКРЯ - Лист1.csv')
data.head()

,column 1,column 2,column 3,column 4
0,Кстати,чувиха,,с голубыми волосами клёвая
1,"так приятно, что меня назвали топовой",чувихой,,NaN
2,вы там где вообще топовые,чувихи,,NaN
3,я не только для тебя говорю,чувиха,,NaN
4,Чисто та,чувиха,",","которая блять говно ебаное, мать в"


In [7]:
columns_to_merge = ['column 1', 'column 2', 'column 3', 'column 4']

In [8]:
data['united'] = data[columns_to_merge].apply(
    lambda row: ' '.join(row.values.astype(str)),axis=1
)

In [9]:
output_path = 'НКРЯ.xlsx'
data.to_excel(output_path, index=False)

# Добавление тегов

In [5]:
morph = pymorphy3.MorphAnalyzer()

In [6]:
def process_word(word: str, target_lemma: str) -> str:
    """Обрабатывает слово с учётом дефисов и пунктуации"""
    # Разделяем слово на части: пунктуация_начало + основная_часть + пунктуация_конец
    match = re.match(r'^([^а-яА-Яa-zA-Z]*)(.*?)([^а-яА-Яa-zA-Z]*)$', word)
    if not match:
        return word

    punct_before, word_part, punct_after = match.groups()

    if not word_part:
        return word

    # Обработка слов через дефис
    if '-' in word_part:
        parts = word_part.split('-')
        processed_parts = []

        for part in parts:
            if not part:  # Пустая часть между дефисами
                processed_parts.append('')
                continue

            try:
                parsed = morph.parse(part)[0]
                if parsed.normal_form == target_lemma:
                    processed_parts.append(f"[TGT]{part}[/TGT]")
                else:
                    processed_parts.append(part)
            except:
                processed_parts.append(part)

        marked_word = '-'.join(processed_parts)
        return f"{punct_before}{marked_word}{punct_after}"

    # Обработка обычных слов
    try:
        parsed = morph.parse(word_part)[0]
        if parsed.normal_form == target_lemma:
            return f"{punct_before}[TGT]{word_part}[/TGT]{punct_after}"
    except:
        pass

    return word

In [7]:
process_word("бро", "бро")

'[TGT]бро[/TGT]'

In [8]:
def process_text(text: str, target_lemma: str, tokenize: bool = True) -> str:
    """Обрабатывает текст, размечая целевые слова"""
    if not isinstance(text, str):
        return text

    if tokenize:
        words = text.split()
    else:
        words = text

    marked_words = [process_word(word, target_lemma) for word in words]

    return " ".join(marked_words) if tokenize else marked_words

In [9]:
def mark_target_words_in_df(
    df: pd.DataFrame,
    text_column: str,
    target_lemma: str,
    output_column: str = 'marked_text',
    inplace: bool = False,
    tokenize: bool = True
) -> Union[pd.DataFrame, None]:

    processor = lambda text: process_text(text, target_lemma, tokenize)

    if inplace:
        df[output_column] = df[text_column].apply(processor)
        return None
    return df.assign(**{output_column: df[text_column].apply(processor)})

## Дед

In [ ]:
data = pd.read_csv('/content/Разметка - дед.csv')
data

,Full context,1 level,2 level,conn
0,". Вероника Михальчук, Пинск, Беларусь: ""Мой пр...",5,9,neutral
1,"Снимать подобное с названием ""Сталинград""-; эт...",5,9,neutral
2,"Марина Аверина, Курджиново, Россия: ""Фильм оск...",5,9,neutral
3,"Коли деды клюкву одну едят, Скулы внукам сведе...",5,9,neutral
4,Да я понимаю что эт не трудно на этапе Инадзум...,5,9,neutral
...,...,...,...,...
295,"Нацепив на морду старый противогаз, который по...",5,9,neutral
296,Своего деда по крови (отца моего отца) я не ви...,5,9,neutral
297,Этот страх был связан с прошлым опытом его дед...,5,9,neutral
298,"Не жалея себя, без колебаний жертвуя своей жиз...",5,9,neutral


In [ ]:
result_df = mark_target_words_in_df(
    df=data,
    text_column='Full context',
    target_lemma='дед',
    output_column='marked_text'
)


In [ ]:
result_df

,Full context,1 level,2 level,conn,marked_text
0,". Вероника Михальчук, Пинск, Беларусь: ""Мой пр...",5,9,neutral,". Вероника Михальчук, Пинск, Беларусь: ""Мой пр..."
1,"Снимать подобное с названием ""Сталинград""-; эт...",5,9,neutral,"Снимать подобное с названием ""Сталинград""-; эт..."
2,"Марина Аверина, Курджиново, Россия: ""Фильм оск...",5,9,neutral,"Марина Аверина, Курджиново, Россия: ""Фильм оск..."
3,"Коли деды клюкву одну едят, Скулы внукам сведе...",5,9,neutral,"Коли [TGT]деды[/TGT] клюкву одну едят, Скулы в..."
4,Да я понимаю что эт не трудно на этапе Инадзум...,5,9,neutral,Да я понимаю что эт не трудно на этапе Инадзум...
...,...,...,...,...,...
295,"Нацепив на морду старый противогаз, который по...",5,9,neutral,"Нацепив на морду старый противогаз, который по..."
296,Своего деда по крови (отца моего отца) я не ви...,5,9,neutral,Своего [TGT]деда[/TGT] по крови (отца моего от...
297,Этот страх был связан с прошлым опытом его дед...,5,9,neutral,Этот страх был связан с прошлым опытом его [TG...
298,"Не жалея себя, без колебаний жертвуя своей жиз...",5,9,neutral,"Не жалея себя, без колебаний жертвуя своей жиз..."


In [ ]:
output_path = 'результат_разметки.xlsx'
result_df.to_excel(output_path, index=False)

## Бро

In [ ]:
bro_df = pd.read_csv('/content/Разметка - бро.csv')
bro_df

,Full context,1 level,2 level,conn
0,Ведь именно там 31 января 1994 года родился из...,1,1,neutral
1,Эта бро реально несколько раз возвращалась в п...,1,1,neutral
2,Бро реально кайфует от написания не то что я,1,1,neutral
3,"Есть некоторые бро, которые постоянно истерят,...",2,2,negative
4,"Бро, это уже в зависимости от требований польз...",3,3,neutral
...,...,...,...,...
295,"лучше вообще расслабится и не ждать, а то так ...",5,10,neutral
296,"Йуухуу, наконец то новый видос Мой бро @nikita...",5,10,neutral
297,"Это ебать погоня. Яныч говорит бро , чтобы тот...",5,10,neutral
298,", когда узнают, что ты их не запомнил. Даже у ...",5,10,neutral


In [ ]:
bro_result_df = mark_target_words_in_df(
    df=bro_df,
    text_column='Full context',
    target_lemma='бро',
    output_column='marked_text'
)

In [ ]:
output_path = 'бро_с_тегами.xlsx'
bro_result_df.to_excel(output_path, index=False)

## Дева

In [ ]:
virgin_df = pd.read_csv('/content/Разметка - дева.csv')
virgin_df

,Full context,1 level,2 level,conn
0,"""По гороскопу"" ""говорят"""", что 23 сентября рож...",1,1,positive
1,Но тема не только про дев но так же и про мужчин.,1,1,neutral
2,в архивах дичайший компромат на нее. То ли не ...,1,1,neutral
3,"Девы, что лучше май-слинг или эрго?",3,3,neutral
4,"Девы, что скажите про этот?? "" July_ ch, это н...",3,3,neutral
...,...,...,...,...
264,Одной дев 8 двоек поставила,7,15,neutral
265,Просто в художке дева с аллергией и глазами,7,15,neutral
266,"Момоа восхитителен и без бороды, Тимоти Шаламе...",7,15,positive
267,"буб (ы) ны, тарелочки гремять краснаи девы и б...",7,15,positive


In [ ]:
virgin_result_df = mark_target_words_in_df(
    df=virgin_df,
    text_column='Full context',
    target_lemma='дева',
    output_column='marked_text'
)

In [ ]:
output_path = 'дева_с_тегами.xlsx'
virgin_result_df.to_excel(output_path, index=False)

## Деваха

In [19]:
girl_again_df = pd.read_csv('/content/деваха - Лист1.csv')
girl_again_df

,Full context,1 level,2 level,conn
0,"Найдём мы тебе деваху , не переживай",5,11,neutral
1,прикольный десантник) деваха такая грудастая,6,12,positive
2,"Я хз, мне больше кажется, что девахе просто г...",6,12,neutral
3,потом мне начала писать какая-то агрессивная д...,6,12,negative
4,усердно порол привязанную к лавке молодую дева...,6,12,neutral
...,...,...,...,...
295,"красивая, спортивная и до предела наглая девах...",6,12,negative
296,"Мне невольно вспоминается одна ""креативная"" де...",6,12,negative
297,"жена, мясистая и медлительная 18-летняя деваха...",6,12,neutral
298,Описание видеоролика: Юная деваха подходит к н...,6,12,neutral


In [20]:
girl_again_result_df = mark_target_words_in_df(
    df=girl_again_df,
    text_column='Full context',
    target_lemma='деваха',
    output_column='marked_text'
)

In [21]:
output_path = 'деваха_с_тегами.xlsx'
girl_again_result_df.to_excel(output_path, index=False)

## Девочка

In [ ]:
girl_df = pd.read_csv('/content/Разметка - девочка.csv')
girl_df

,Full context,1 level,2 level,conn
0,"И, указав на девочку: ""МА-ЛЕНЬ-КАЯ ДЕ-ВУ-ШКА"".",6,12,neutral
1,"Маленькая девочка, на свой День рождения, поже...",6,12,neutral
2,"Для девочки, видимо, это вошло в привычку и он...",6,12,neutral
3,"Это продолжалось целый год, и сложилось так, ч...",6,12,neutral
4,"Просто спальня маленькой девочки, а не ""праздн...",6,12,neutral
...,...,...,...,...
295,И кстати лишь одна девочка может меня по имени...,7,15,neutral
296,И я все равно благодарна за бесценный опыт и з...,6,12,neutral
297,Хотя я не прошу формулировать мысли 12ух летне...,6,12,neutral
298,"Девочки с Воронежа, если увидите его скажите м...",3,3,neutral


In [ ]:
girl_result_df = mark_target_words_in_df(
    df=girl_df,
    text_column='Full context',
    target_lemma='девочка',
    output_column='marked_text'
)

In [ ]:
output_path = 'девочка_с_тегами.xlsx'
girl_result_df.to_excel(output_path, index=False)

## Женщина

In [ ]:
woman_df = pd.read_csv('/content/Разметка - женщина.csv')
woman_df

,Full context,1 level,2 level,conn
0,Если в отношениях женщина сильнее мужчины ― эт...,1,1,neutral
1,"Проблема не в том, что она решает что, когда и...",8,8,neutral
2,"Избавляйтесь от слабых мальчиков, это ненужный...",1,1,neutral
3,"В местной газете была заметка о том, как 83-ле...",6,14,neutral
4,"Вместо того, чтобы напоминать беременным, чтоб...",1,1,neutral
...,...,...,...,...
295,"Такого человека уже нет, и жизни тоже нет: люб...",5,11,neutral
296,"Я, кстати, давно заметил, что женщины, описыва...",1,1,neutral
297,Мужчина не сможет научиться действовать в свои...,1,1,neutral
298,"Любой мужчина найдет в своей истории случаи, к...",1,1,neutral


In [ ]:
woman_result_df = mark_target_words_in_df(
    df=woman_df,
    text_column='Full context',
    target_lemma='женщина',
    output_column='marked_text'
)

In [ ]:
output_path = 'женщина_с_тегами.xlsx'
woman_result_df.to_excel(output_path, index=False)

## Мальчик

In [ ]:
boy_df = pd.read_csv('/content/Разметка - мальчик.csv')
boy_df

,Full context,1 level,2 level,conn
0,Шпионка-танцовщица Мата Хари послала целящимся...,3,3,neutral
1,"Например, мальчик Иоанн Креститель на известно...",6,12,neutral
2,В «Тайной вечере» можно разглядеть перевёрнуту...,6,12,neutral
3,Гороскоп: мальчикам о девочках. ДЕВУШКА-БЛИЗН...,1,1,neutral
4,"Этих мальчиков и девочек, работающих сейчас в ...",7,15,neutral
...,...,...,...,...
295,"Мальчик обернулся, взял свои вещи и понуро пош...",6,12,neutral
296,— спросил Алек и снова взглянул на мальчика.,6,12,neutral
297,И когда мальчик обратил на него яркие от испуг...,6,12,neutral
298,Именно в этом возрасте к вашему сыну приходит ...,1,1,neutral


In [ ]:
boy_result_df = mark_target_words_in_df(
    df=boy_df,
    text_column='Full context',
    target_lemma='мальчик',
    output_column='marked_text'
)

In [ ]:
output_path = 'мальчик_с_тегами.xlsx'
boy_result_df.to_excel(output_path, index=False)

## Мужчина

In [ ]:
man_df = pd.read_csv('/content/Разметка - мужчина.csv')
man_df

,Full context,1 level,2 level,conn
0,А это потому что ты привыкла помогать да напра...,5,11,neutral
1,"Тебе мужчина нужен тебе под стать, состоятельн...",5,11,positive
2,"– Есть такие мужчины на свете, даже и немало.",6,13,neutral
3,"Нам нужен друг и соратник, мужчина, близкий по...",1,1,neutral
4,"Стоял перед ней не кот, а Мужчина Ее Мечты: ни...",5,11,positive
...,...,...,...,...
295,"Опрос показал, что почти 40% мужчин в России п...",1,1,neutral
296,"К брюнеткам мужчины относятся более серьезно, ...",1,1,neutral
297,Исследование Аугсбургского университета (Минне...,1,1,neutral
298,62% сверхбогатых мужчин мира выбрали в качеств...,1,1,neutral


In [ ]:
man_result_df = mark_target_words_in_df(
    df=man_df,
    text_column='Full context',
    target_lemma='мужчина',
    output_column='marked_text'
)

In [ ]:
output_path = 'мужчина_с_тегами.xlsx'
man_result_df.to_excel(output_path, index=False)

## Пацан

In [ ]:
guy_df = pd.read_csv('/content/Разметка - пацан.csv')
guy_df

,Full context,1 level,2 level,conn,Unnamed: 4
0,А вот пацаны исправно ходят на пеленку.,6,12,neutral,NaN
1,", будешь в северном звякни! Пацаны скучают! ...",5,10,neutral,NaN
2,Ведь в каком - то случае собака не за задницу ...,6,12,neutral,NaN
3,Пацаны) А у Вас тут жара) Вступаю в ваш клуб!!,3,3,positive,NaN
4,"так держать, пацаны не плачут, а терпят!",8,8,neutral,NaN
...,...,...,...,...,...
295,"Что пацаны, 1, 9к на Али?",3,3,neutral,NaN
296,"Изнуренные жарой и жаждой, мы с пацанами решил...",5,10,neutral,NaN
297,"В данном случае у пацана Белоруссия, зачем ты ...",7,15,neutral,NaN
298,"Тут пацан рядом, орет мне: «Ща помогу!»",7,15,neutral,NaN


In [ ]:
guy_result_df = mark_target_words_in_df(
    df=guy_df,
    text_column='Full context',
    target_lemma='пацан',
    output_column='marked_text'
)

In [ ]:
output_path = 'пацан_с_тегами.xlsx'
guy_result_df.to_excel(output_path, index=False)

## Чувиха

In [10]:
dude_df = pd.read_csv('/content/Чувиха - Лист1.csv')
dude_df

,Full context,1 level,2 level,conn
0,Кстати чувиха с голубыми волосами клёвая,7,15,positive
1,"так приятно, что меня назвали топовой чувихой",7,15,positive
2,вы там где вообще топовые чувихи,3,3,positive
3,я не только для тебя говорю чувиха,3,3,neutral
4,"Чисто та чувиха , которая блять говно ебаное",7,15,negative
...,...,...,...,...
295,"Звонил вам, но вы не отвечали. Ну а о том что ...",7,15,neutral
296,Чувиха ! Дом 2 это говно. Ненавижу этот тупой ...,3,3,neutral
297,"Его напарница – классная чувиха , игравшая вме...",7,15,positive
298,"Сидит чувак (или, возможно, чувиха ) и что-то ...",1,1,neutral


In [11]:
dude_result_df = mark_target_words_in_df(
    df=dude_df,
    text_column='Full context',
    target_lemma='чувиха',
    output_column='marked_text'
)

In [12]:
output_path = 'чувиха_с_тегами.xlsx'
dude_result_df.to_excel(output_path, index=False)

## Юнец

In [16]:
young_df = pd.read_csv('/content/Юнец - Лист1.csv')
young_df

,Full context,1 level,2 level,conn
0,Всякие юнцы приходили ко мне с просьбой,6,12,neutral
1,тридцатилетней парочки либо же у безусых юнцов,6,12,neutral
2,Ханжа со стажем и юнец без стажа,6,12,neutral
3,А рядом ты ― совсем ещё юнец,6,12,neutral
4,"его на землю и сказал: ""Глупый юнец , ты что, ...",6,12,negative
...,...,...,...,...
295,Когда эгоистичный юнец вернулся к своему,6,12,negative
296,Илья положил руку на плечо разбушевавшегося юн...,6,12,negative
297,"Возможно, он был пятнадцатилетним юнцом , как ...",6,12,neutral
298,"А мы снова, как зеленые юнцы , кричим, что ста...",6,12,neutral


In [17]:
young_result_df = mark_target_words_in_df(
    df=young_df,
    text_column='Full context',
    target_lemma='юнец',
    output_column='marked_text'
)

In [18]:
output_path = 'юнец_с_тегами.xlsx'
young_result_df.to_excel(output_path, index=False)

# Добавление тегов для лексем, которые не распознает pymorphy

In [ ]:
# Словарь сленговых слов и их  форм
SLANG_DICT = {
    'чел': ['чел', 'чела', 'челу', 'челом', 'челе', 'челы', 'челам', 'челов', 'челами', 'челах'],
    'челик': ['челик', 'челика', 'челику', 'челиком', 'челике', 'челики', 'челиков', 'челикам', 'челиками', 'челиках'],
    'мэн': ['мэн', 'мэна', 'мэну', 'мэном', 'мэне', 'мэны', 'мэнов','мэнам', 'мэнов', 'мэнами', 'мэнах', 'менов', 'меном', 'мен'],
    'герл': ['герлу', 'гэрлы', 'герлы', 'гёрла', 'герла', 'герлам', 'герл', 'гёрл', 'гёрлу', 'герлс', 'гёрлс', 'герлами', 'гёрлы', 'герлов']
}
SLANG_FORMS = {form: base for base, forms in SLANG_DICT.items() for form in forms}

def get_slang_base_form(word: str) -> Union[str, None]:
    """Находит базовую форму сленгового слова по словарю"""
    return SLANG_FORMS.get(word.lower())

def process_word(word: str, target_lemma: str) -> str:
    """Обрабатывает слово с учётом сленга и пунктуации"""
    # Разделяем слово и пунктуацию
    match = re.match(r'^([^а-яА-Яa-zA-Z]*)(.*?)([^а-яА-Яa-zA-Z]*)$', word)
    if not match:
        return word
    punct_before, word_part, punct_after = match.groups()

    if not word_part:
        return word
    # Проверяем, является ли слово сленгом
    base_form = get_slang_base_form(word_part)

    # Если это целевая лексема или её форма
    if base_form == target_lemma:
        return f"{punct_before}[TGT]{word_part}[/TGT]{punct_after}"

    # Обработка составных слов через дефис
    if '-' in word_part:
        parts = word_part.split('-')
        marked_parts = [
            f"[TGT]{part}[/TGT]" if get_slang_base_form(part) == target_lemma else part
            for part in parts
        ]
        return f"{punct_before}{'-'.join(marked_parts)}{punct_after}"

    return word

def process_text(text: str, target_lemma: str) -> str:
    """Обрабатывает текст, сохраняя пунктуацию"""
    words = re.findall(r'\S+', text)
    marked_words = [process_word(word, target_lemma) for word in words]
    return ' '.join(marked_words)

def mark_slang_words_in_df(
    df: pd.DataFrame,
    text_column: str,
    target_lemma: str,
    output_column: str = 'marked_text',
    inplace: bool = False
) -> Union[pd.DataFrame, None]:
    """
    Разметка сленговых слов по пользовательскому словарю
    """
    if target_lemma not in SLANG_DICT:
        raise ValueError(f"Слово '{target_lemma}' отсутствует в словаре сленга. Добавьте его в SLANG_LEXICON.")

    processor = lambda text: process_text(str(text), target_lemma)

    if inplace:
        df[output_column] = df[text_column].apply(processor)
        return None
    return df.assign(**{output_column: df[text_column].apply(processor)})

## Чел

In [ ]:
chel_df = pd.read_csv('/content/Разметка - чел.csv')
chel_df

FileNotFoundError: [Errno 2] No such file or directory: '/content/Разметка - чел.csv'

In [ ]:
chel_result_df = mark_slang_words_in_df(chel_df, 'Full context', 'чел')
chel_result_df

,Full context,1 level,2 level,conn,marked_text
0,"долбаеб Китай это кнр, а чел говорит о северно...",1,1,neutral,"долбаеб Китай это кнр, а [TGT]чел[/TGT] говори..."
1,Бородатый чел Мне нравится,1,1,neutral,Бородатый [TGT]чел[/TGT] Мне нравится
2,"Че, где? Там чел сам всё сделал.",1,1,neutral,"Че, где? Там [TGT]чел[/TGT] сам всё сделал."
3,"Ну сорян, чел просто высирает рандомную хуету ...",1,1,negative,"Ну сорян, [TGT]чел[/TGT] просто высирает рандо..."
4,Или чел военный с шизой,1,1,negative,Или [TGT]чел[/TGT] военный с шизой
...,...,...,...,...,...
295,"Они ничем не хуже и не уродливые, в отличие от...",3,3,neutral,"Они ничем не хуже и не уродливые, в отличие от..."
296,"Чел, ты реально поверил? ) Это я мем просто сд...",3,3,neutral,"[TGT]Чел[/TGT], ты реально поверил? ) Это я ме..."
297,ой господи чел ты серьезно?,3,3,negative,ой господи [TGT]чел[/TGT] ты серьезно?
298,"Чел, это было ещё до всей этой моды на толерас...",3,3,neutral,"[TGT]Чел[/TGT], это было ещё до всей этой моды..."


In [ ]:
output_path = 'чел_с_тегами.xlsx'
chel_result_df.to_excel(output_path, index=False)

## Челик

In [ ]:
chelik_df = pd.read_csv('/content/Разметка - челик.csv')
chelik_df

,Full context,1 level,2 level,conn
0,Видела челика который за год французкий до б2 ...,1,1.0,neutral
1,"Забивной челик, скрывается под видом девушки, ...",1,1.0,neutral
2,"Хз, был смешной челик",1,1.0,positive
3,"Сказал челик, который пишет одно и то же, прип...",1,1.0,NaN
4,"А еще я любила общаться с челиками, говорить, ...",1,1.0,neutral
...,...,...,...,...
295,"Да у челика, видать, подготовка к этой параше ...",6,12.0,negative
296,Я в школе с челиком так развлекались Становили...,6,12.0,neutral
297,"У меня два варианта, либо ему гдето 18-20 лет,...",6,12.0,neutral
298,"эпл настолько топ, что дефолт провода не юзают...",1,NaN,negative


In [ ]:
chelik_result_df = mark_slang_words_in_df(chelik_df, 'Full context', 'челик')
chelik_result_df

,Full context,1 level,2 level,conn,marked_text
0,Видела челика который за год французкий до б2 ...,1,1.0,neutral,Видела [TGT]челика[/TGT] который за год францу...
1,"Забивной челик, скрывается под видом девушки, ...",1,1.0,neutral,"Забивной [TGT]челик[/TGT], скрывается под видо..."
2,"Хз, был смешной челик",1,1.0,positive,"Хз, был смешной [TGT]челик[/TGT]"
3,"Сказал челик, который пишет одно и то же, прип...",1,1.0,NaN,"Сказал [TGT]челик[/TGT], который пишет одно и ..."
4,"А еще я любила общаться с челиками, говорить, ...",1,1.0,neutral,"А еще я любила общаться с [TGT]челиками[/TGT],..."
...,...,...,...,...,...
295,"Да у челика, видать, подготовка к этой параше ...",6,12.0,negative,"Да у [TGT]челика[/TGT], видать, подготовка к э..."
296,Я в школе с челиком так развлекались Становили...,6,12.0,neutral,Я в школе с [TGT]челиком[/TGT] так развлекалис...
297,"У меня два варианта, либо ему гдето 18-20 лет,...",6,12.0,neutral,"У меня два варианта, либо ему гдето 18-20 лет,..."
298,"эпл настолько топ, что дефолт провода не юзают...",1,NaN,negative,"эпл настолько топ, что дефолт провода не юзают..."


In [ ]:
output_path = 'челик_с_тегами.xlsx'
chelik_result_df.to_excel(output_path, index=False)

## Герл

In [ ]:
girl_rus_df = pd.read_csv('/content/Разметка - герл.csv')
girl_rus_df

,Full context,1 level,2 level,conn
0,мы лицезреем двух главных героев: 14-летнюю ге...,6,12,neutral
1,"Ей 9 лет и она тайком курила,, типо депрессивн...",6,12,negative
2,"В открытом космосе, подлетая к кораблю на дырк...",7,15,negative
3,нее не повелся ибо она 80 кг весит и особо не ...,7,15,negative
4,"там какой-то чат, в нем видимо ихняя косоглаза...",7,15,negative
...,...,...,...,...
261,А еще это чудо автомобильной мысли быстро прев...,7,15,positive
262,"Таня, ты клевая герла) А дальше все закружилос...",7,15,positive
263,"Левую штанину закатать до колена (Стинг, Агути...",7,15,positive
264,Бонжур пацантре и моднейшие герлы! Хотим позн...,3,3,positive


In [ ]:
to_add = pd.read_csv('/content/Добавить в разметку - Лист1.csv')
to_add

,Full context,1 level,2 level,conn
0,Скоро ты станешь эмо депрешн герл,1,1,neutral
1,".А как рядом прошли две девченки типа ""эмо - г...",1,1,neutral
2,Как в голове у тумблер- гёрл,1,1,neutral
3,". Во-вторых, не консьержка, а ресепшн герл .",1,1,neutral
4,"тайского языка слились в одну, то ли сервис ге...",1,1,neutral
5,исполняется нашей прекрасной и удивительной Дж...,1,1,positive
6,в прошлом. Сейчас в моде – дерзкие тумблер гер...,1,1,positive
7,душевную натуру приверженцев эмо. Макияж эмо -...,1,1,neutral
8,""" Принципы стиля "" Одеваемся в стиле Тумблер Г...",1,1,neutral
9,виде девушки стали называться тумблер герл . Т...,1,1,neutral


In [ ]:
girl_rus_result_df = mark_slang_words_in_df(girl_rus_df, 'Full context', 'герл')
girl_rus_result_df

NameError: name 'girl_rus_df' is not defined

In [ ]:
to_add_result_df = mark_slang_words_in_df(to_add, 'Full context', 'герл')
to_add_result_df

,Full context,1 level,2 level,conn,marked_text
0,Скоро ты станешь эмо депрешн герл,1,1,neutral,Скоро ты станешь эмо депрешн [TGT]герл[/TGT]
1,".А как рядом прошли две девченки типа ""эмо - г...",1,1,neutral,".А как рядом прошли две девченки типа ""эмо - [..."
2,Как в голове у тумблер- гёрл,1,1,neutral,Как в голове у тумблер- [TGT]гёрл[/TGT]
3,". Во-вторых, не консьержка, а ресепшн герл .",1,1,neutral,". Во-вторых, не консьержка, а ресепшн [TGT]гер..."
4,"тайского языка слились в одну, то ли сервис ге...",1,1,neutral,"тайского языка слились в одну, то ли сервис [T..."
5,исполняется нашей прекрасной и удивительной Дж...,1,1,positive,исполняется нашей прекрасной и удивительной Дж...
6,в прошлом. Сейчас в моде – дерзкие тумблер гер...,1,1,positive,в прошлом. Сейчас в моде – дерзкие тумблер [TG...
7,душевную натуру приверженцев эмо. Макияж эмо -...,1,1,neutral,душевную натуру приверженцев эмо. Макияж эмо -...
8,""" Принципы стиля "" Одеваемся в стиле Тумблер Г...",1,1,neutral,""" Принципы стиля "" Одеваемся в стиле Тумблер [..."
9,виде девушки стали называться тумблер герл . Т...,1,1,neutral,виде девушки стали называться тумблер [TGT]гер...


In [ ]:
output_path = 'герл_остатки.xlsx'
to_add_result_df.to_excel(output_path, index=False)

## Мэн

In [ ]:
man_rus_df = pd.read_csv('/content/Разметка - мэн.csv')
man_rus_df

,Full context,1 level,2 level,conn
0,"нужно же по кому то страдать, мэн",3,3,neutral
1,ты знаком эти четыре буквы мэн,3,3,neutral
2,"Соре Мэн, не заметил последнее сообщение",3,3,neutral
3,"Ну, мэн, у меня четвёртые переключают в любом ...",3,3,neutral
4,"6 кадров это топ, мэн!! Ты не представляешь к...",3,3,positive
...,...,...,...,...
295,Что у меня постоянно новый мэн,5,11,neutral
296,"мэн пытался сдать комнату, ему написали на ави...",7,15,neutral
297,Опять моего мэна затрата,5,11,neutral
298,там для мэнов платно?,1,1,neutral


In [ ]:
man_rus_result_df = mark_slang_words_in_df(man_rus_df, 'Full context', 'мэн')
man_rus_result_df

,Full context,1 level,2 level,conn,marked_text
0,"нужно же по кому то страдать, мэн",3,3,neutral,"нужно же по кому то страдать, [TGT]мэн[/TGT]"
1,ты знаком эти четыре буквы мэн,3,3,neutral,ты знаком эти четыре буквы [TGT]мэн[/TGT]
2,"Соре Мэн, не заметил последнее сообщение",3,3,neutral,"Соре [TGT]Мэн[/TGT], не заметил последнее сооб..."
3,"Ну, мэн, у меня четвёртые переключают в любом ...",3,3,neutral,"Ну, [TGT]мэн[/TGT], у меня четвёртые переключа..."
4,"6 кадров это топ, мэн!! Ты не представляешь к...",3,3,positive,"6 кадров это топ, [TGT]мэн[/TGT]!! Ты не предс..."
...,...,...,...,...,...
295,Что у меня постоянно новый мэн,5,11,neutral,Что у меня постоянно новый [TGT]мэн[/TGT]
296,"мэн пытался сдать комнату, ему написали на ави...",7,15,neutral,"[TGT]мэн[/TGT] пытался сдать комнату, ему напи..."
297,Опять моего мэна затрата,5,11,neutral,Опять моего [TGT]мэна[/TGT] затрата
298,там для мэнов платно?,1,1,neutral,там для [TGT]мэнов[/TGT] платно?


In [ ]:
output_path = 'мэн_с_тегами.xlsx'
man_rus_result_df.to_excel(output_path, index=False)

# Склейка табличек в один датафрейм

В этом коде для склейки не хватает 3 датафреймов для слов юнец, деваха и чувиха, так как они были добавлены в выборку позднее, чем остальные слова. Предобработанные контексты с этими словами были вручную добавлену в выборку позже

In [ ]:
combined_df = pd.concat([result_df, bro_result_df, virgin_result_df,
                         girl_result_df, woman_result_df, boy_result_df,
                         man_result_df, guy_result_df, man_rus_result_df,
                         girl_rus_result_df, chel_result_df, chelik_result_df],
                        axis=0, ignore_index=True)
combined_df = combined_df.sample(frac=1).reset_index(drop=True)

,Full context,1 level,2 level,conn,marked_text,Unnamed: 4
0,". Вероника Михальчук, Пинск, Беларусь: ""Мой пр...",5,9.0,neutral,". Вероника Михальчук, Пинск, Беларусь: ""Мой пр...",NaN
1,"Снимать подобное с названием ""Сталинград""-; эт...",5,9.0,neutral,"Снимать подобное с названием ""Сталинград""-; эт...",NaN
2,"Марина Аверина, Курджиново, Россия: ""Фильм оск...",5,9.0,neutral,"Марина Аверина, Курджиново, Россия: ""Фильм оск...",NaN
3,"Коли деды клюкву одну едят, Скулы внукам сведе...",5,9.0,neutral,"Коли [TGT]деды[/TGT] клюкву одну едят, Скулы в...",NaN
4,Да я понимаю что эт не трудно на этапе Инадзум...,5,9.0,neutral,Да я понимаю что эт не трудно на этапе Инадзум...,NaN
...,...,...,...,...,...,...
3530,"Да у челика, видать, подготовка к этой параше ...",6,12.0,negative,"Да у [TGT]челика[/TGT], видать, подготовка к э...",NaN
3531,Я в школе с челиком так развлекались Становили...,6,12.0,neutral,Я в школе с [TGT]челиком[/TGT] так развлекалис...,NaN
3532,"У меня два варианта, либо ему гдето 18-20 лет,...",6,12.0,neutral,"У меня два варианта, либо ему гдето 18-20 лет,...",NaN
3533,"эпл настолько топ, что дефолт провода не юзают...",1,NaN,negative,"эпл настолько топ, что дефолт провода не юзают...",NaN


In [ ]:
output_path = 'Весь датасет с тегами.xlsx'
combined_df.to_excel(output_path, index=False)

# Собственно предобработка

In [3]:
df = pd.read_excel('/content/Датасет.xlsx')
df.head()

,Full context,1 level,2 level,conn,marked_text
0,". Вероника Михальчук, Пинск, Беларусь: ""Мой пр...",5,9,neutral,". Вероника Михальчук, Пинск, Беларусь: ""Мой пр..."
1,"Снимать подобное с названием ""Сталинград""-; эт...",5,9,neutral,"Снимать подобное с названием ""Сталинград""-; эт..."
2,"Марина Аверина, Курджиново, Россия: ""Фильм оск...",5,9,neutral,"Марина Аверина, Курджиново, Россия: ""Фильм оск..."
3,"Коли деды клюкву одну едят, Скулы внукам сведе...",5,9,neutral,"Коли [TGT]деды[/TGT] клюкву одну едят, Скулы в..."
4,Да я понимаю что эт не трудно на этапе Инадзум...,5,9,neutral,Да я понимаю что эт не трудно на этапе Инадзум...


In [15]:
#перемешиваю примеры
mixed_df = df.sample(frac=1).reset_index(drop=True)
mixed_df

,Full context,1 level,2 level,conn,marked_text
0,Ща погодите как это 100 челиков лимит,2,2,neutral,Ща погодите как это 100 [TGT]челиков[/TGT] лимит
1,"карабинеру, что это действительно моя гёрл , п...",5,11,neutral,"карабинеру, что это действительно моя [TGT]гёр..."
2,"Знавал я дев , которым и отсутствия руки, не м...",7,15,neutral,"Знавал я [TGT]дев[/TGT] , которым и отсутствия..."
3,маза поднять бабла! Am Мне тут позвонила одна ...,7,15,neutral,маза поднять бабла! Am Мне тут позвонила одна ...
4,"Забивной челик, скрывается под видом девушки, ...",1,1,neutral,"Забивной [TGT]челик[/TGT], скрывается под видо..."
...,...,...,...,...,...
3564,Сочувствую чел твоему телеку,3,3,neutral,Сочувствую [TGT]чел[/TGT] твоему телеку
3565,"На одном далеком районе, в одном далеком дворе...",6,12,neutral,"На одном далеком районе, в одном далеком дворе..."
3566,Ето аватарка одному челику,1,1,neutral,Ето аватарка одному [TGT]челику[/TGT]
3567,". Думает, раз с бумажкой, то крутой. Короче, г...",3,3,neutral,". Думает, раз с бумажкой, то крутой. Короче, [..."


In [23]:
def preprocess(text):
    # Удаление ссылок
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Удаление слэшей
    text = re.sub(r'\/', ' ', text)
    # Удаление цифр
    text = re.sub(r'\d+', '', text)
    return text

In [30]:
#Применяю предобработку к датасету
preprocessed_df = df.copy()
preprocessed_df['marked_text'] = preprocessed_df['marked_text'].apply(lambda x: pd.Series(preprocess(x)))
preprocessed_df

,Full context,1 level,2 level,conn,marked_text
0,"хехе, чел, у неё родичи в мусарне, соответстве...",3,3,neutral,"хехе, [TGT]чел[/TGT], у неё родичи в мусарне, ..."
1,МНОГО и ПОКА ... ¶ aleksey ты очень агрессивны...,7,15,negative,много и пока ... aleksey ты очень агрессивный ...
2,"входящего. Стоит двери приоткрыться, как гёрл ...",7,15,neutral,"входящего. стоит двери приоткрыться, как [TGT]..."
3,"С момента создания ретро-инструкции о том, как...",1,1,neutral,"с момента создания ретро-инструкции о том, как..."
4,Жила-была девочка.,6,12,neutral,жила-была [TGT]девочка[/TGT].
...,...,...,...,...,...
3564,"Иначе говоря, герой нам представлен как самый-...",7,15,neutral,"иначе говоря, герой нам представлен как самый-..."
3565,"ЛЮДИ,,,,, НАЙДИТЕ ПОЖАЛУЙСТА БРАТА, ДИМУ, ЧЕРН...",7,15,neutral,"люди,,,,, найдите пожалуйста брата, диму, черн..."
3566,"У меня там ни кого нет(((, а книги уйдут, уже ...",1,1,positive,"у меня там ни кого нет(((, а книги уйдут, уже ..."
3567,нее не повелся ибо она кг весит и особо не се...,7,15,negative,нее не повелся ибо она кг весит и особо не се...


Я совершила ошибку, когда сначала добавила в текст теги, а только потом решила его предобработать, следовало поступить наоборот. Но я нашла способ привести текст в нижний регист без приведения к нижнему регистру тегов.

In [24]:
def lowercase_with_fixed_tags(text):
    """
    Приводит весь текст к нижнему регистру, но сохраняет теги [TGT] и [/TGT] без изменений.
    """
    # Шаг 1: Заменяем [TGT] и [/TGT] на метки-заглушки
    text = re.sub(r'\[TGT\]', '___TGT_TAG___', text)
    text = re.sub(r'\[/TGT\]', '___END_TGT_TAG___', text)

    # Шаг 2: Переводим весь текст в нижний регистр
    text = text.lower()

    # Шаг 3: Восстанавливаем оригинальные теги
    text = re.sub(r'___tgt_tag___', '[TGT]', text)
    text = re.sub(r'___end_tgt_tag___', '[/TGT]', text)

    return text

In [32]:
#привожу текст в нижний регист, сохранив теги
preprocessed_df['text_processed'] = preprocessed_df['marked_text'].apply(lowercase_with_fixed_tags)

In [33]:
preprocessed_df

,Full context,1 level,2 level,conn,marked_text,text_processed
0,"хехе, чел, у неё родичи в мусарне, соответстве...",3,3,neutral,"хехе, [TGT]чел[/TGT], у неё родичи в мусарне, ...","хехе, [TGT]чел[/TGT], у неё родичи в мусарне, ..."
1,МНОГО и ПОКА ... ¶ aleksey ты очень агрессивны...,7,15,negative,много и пока ... aleksey ты очень агрессивный ...,много и пока ... aleksey ты очень агрессивный ...
2,"входящего. Стоит двери приоткрыться, как гёрл ...",7,15,neutral,"входящего. стоит двери приоткрыться, как [TGT]...","входящего. стоит двери приоткрыться, как [TGT]..."
3,"С момента создания ретро-инструкции о том, как...",1,1,neutral,"с момента создания ретро-инструкции о том, как...","с момента создания ретро-инструкции о том, как..."
4,Жила-была девочка.,6,12,neutral,жила-была [TGT]девочка[/TGT].,жила-была [TGT]девочка[/TGT].
...,...,...,...,...,...,...
3564,"Иначе говоря, герой нам представлен как самый-...",7,15,neutral,"иначе говоря, герой нам представлен как самый-...","иначе говоря, герой нам представлен как самый-..."
3565,"ЛЮДИ,,,,, НАЙДИТЕ ПОЖАЛУЙСТА БРАТА, ДИМУ, ЧЕРН...",7,15,neutral,"люди,,,,, найдите пожалуйста брата, диму, черн...","люди,,,,, найдите пожалуйста брата, диму, черн..."
3566,"У меня там ни кого нет(((, а книги уйдут, уже ...",1,1,positive,"у меня там ни кого нет(((, а книги уйдут, уже ...","у меня там ни кого нет(((, а книги уйдут, уже ..."
3567,нее не повелся ибо она кг весит и особо не се...,7,15,negative,нее не повелся ибо она кг весит и особо не се...,нее не повелся ибо она кг весит и особо не се...


In [36]:
output_path = 'Совсем окончательный датасет.xlsx'
preprocessed_df.to_excel(output_path, index=False)

Так как,как я уже сказала выше, 3 лексемы были добавлены позже остальных, их я обработала отдельно а затем вручну добавила в общий датасет

In [25]:
df = pd.read_excel('/content/3 главных слова.xlsx')
df.head()

,Full context,1 level,2 level,conn,marked_text
0,Всякие юнцы приходили ко мне с просьбой,6,12,neutral,Всякие [TGT]юнцы[/TGT] приходили ко мне с прос...
1,тридцатилетней парочки либо же у безусых юнцов,6,12,neutral,тридцатилетней парочки либо же у безусых [TGT]...
2,Ханжа со стажем и юнец без стажа,6,12,neutral,Ханжа со стажем и [TGT]юнец[/TGT] без стажа
3,А рядом ты ― совсем ещё юнец,6,12,neutral,А рядом ты ― совсем ещё [TGT]юнец[/TGT]
4,"его на землю и сказал: ""Глупый юнец , ты что, ...",6,12,negative,"его на землю и сказал: ""Глупый [TGT]юнец[/TGT]..."


In [26]:
#удаляю ссылки числа и слэши
preprocessed_df = df.copy()
preprocessed_df['marked_text'] = preprocessed_df['marked_text'].apply(lambda x: pd.Series(preprocess(x)))
preprocessed_df

,Full context,1 level,2 level,conn,marked_text
0,Всякие юнцы приходили ко мне с просьбой,6,12,neutral,Всякие [TGT]юнцы[/TGT] приходили ко мне с прос...
1,тридцатилетней парочки либо же у безусых юнцов,6,12,neutral,тридцатилетней парочки либо же у безусых [TGT]...
2,Ханжа со стажем и юнец без стажа,6,12,neutral,Ханжа со стажем и [TGT]юнец[/TGT] без стажа
3,А рядом ты ― совсем ещё юнец,6,12,neutral,А рядом ты ― совсем ещё [TGT]юнец[/TGT]
4,"его на землю и сказал: ""Глупый юнец , ты что, ...",6,12,negative,"его на землю и сказал: ""Глупый [TGT]юнец[/TGT]..."
...,...,...,...,...,...
895,"красивая, спортивная и до предела наглая девах...",6,12,negative,"красивая, спортивная и до предела наглая [TGT]..."
896,"Мне невольно вспоминается одна ""креативная"" де...",6,12,negative,"Мне невольно вспоминается одна ""креативная"" [T..."
897,"жена, мясистая и медлительная 18-летняя деваха...",6,12,neutral,"жена, мясистая и медлительная 18-летняя [TGT]д..."
898,Описание видеоролика: Юная деваха подходит к н...,6,12,neutral,Описание видеоролика: Юная [TGT]деваха[/TGT] п...


In [27]:
#привожу к нижнему регистру
preprocessed_df['text_processed'] = preprocessed_df['marked_text'].apply(lowercase_with_fixed_tags)

In [28]:
preprocessed_df

,Full context,1 level,2 level,conn,marked_text,text_processed
0,Всякие юнцы приходили ко мне с просьбой,6,12,neutral,Всякие [TGT]юнцы[/TGT] приходили ко мне с прос...,всякие [TGT]юнцы[/TGT] приходили ко мне с прос...
1,тридцатилетней парочки либо же у безусых юнцов,6,12,neutral,тридцатилетней парочки либо же у безусых [TGT]...,тридцатилетней парочки либо же у безусых [TGT]...
2,Ханжа со стажем и юнец без стажа,6,12,neutral,Ханжа со стажем и [TGT]юнец[/TGT] без стажа,ханжа со стажем и [TGT]юнец[/TGT] без стажа
3,А рядом ты ― совсем ещё юнец,6,12,neutral,А рядом ты ― совсем ещё [TGT]юнец[/TGT],а рядом ты ― совсем ещё [TGT]юнец[/TGT]
4,"его на землю и сказал: ""Глупый юнец , ты что, ...",6,12,negative,"его на землю и сказал: ""Глупый [TGT]юнец[/TGT]...","его на землю и сказал: ""глупый [TGT]юнец[/TGT]..."
...,...,...,...,...,...,...
895,"красивая, спортивная и до предела наглая девах...",6,12,negative,"красивая, спортивная и до предела наглая [TGT]...","красивая, спортивная и до предела наглая [TGT]..."
896,"Мне невольно вспоминается одна ""креативная"" де...",6,12,negative,"Мне невольно вспоминается одна ""креативная"" [T...","мне невольно вспоминается одна ""креативная"" [T..."
897,"жена, мясистая и медлительная 18-летняя деваха...",6,12,neutral,"жена, мясистая и медлительная 18-летняя [TGT]д...","жена, мясистая и медлительная 18-летняя [TGT]д..."
898,Описание видеоролика: Юная деваха подходит к н...,6,12,neutral,Описание видеоролика: Юная [TGT]деваха[/TGT] п...,описание видеоролика: юная [TGT]деваха[/TGT] п...


In [29]:
output_path = '3 окончательный датасет.xlsx'
preprocessed_df.to_excel(output_path, index=False)

Здесь перемешиваю уже полностью сформированный датасет

In [33]:
df = pd.read_excel('/content/Совсем окончательный датасет.xlsx')
df

,Full context,1 level,2 level,conn,marked_text
0,"хехе, чел, у неё родичи в мусарне, соответстве...",3,3,neutral,"хехе, [TGT]чел[/TGT], у неё родичи в мусарне, ..."
1,МНОГО и ПОКА ... ¶ aleksey ты очень агрессивны...,7,15,negative,много и пока ... aleksey ты очень агрессивный ...
2,"входящего. Стоит двери приоткрыться, как гёрл ...",7,15,neutral,"стоит двери приоткрыться, как [TGT]гёрл[/TGT] ..."
3,"С момента создания ретро-инструкции о том, как...",1,1,neutral,"с момента создания ретро-инструкции о том, как..."
4,Жила-была девочка.,6,12,neutral,жила-была [TGT]девочка[/TGT].
...,...,...,...,...,...
4463,"красивая, спортивная и до предела наглая девах...",6,12,negative,"красивая, спортивная и до предела наглая [TGT]..."
4464,"Мне невольно вспоминается одна ""креативная"" де...",6,12,negative,"мне невольно вспоминается одна ""креативная"" [T..."
4465,"жена, мясистая и медлительная 18-летняя деваха...",6,12,neutral,"жена, мясистая и медлительная 18-летняя [TGT]д..."
4466,Описание видеоролика: Юная деваха подходит к н...,6,12,neutral,описание видеоролика: юная [TGT]деваха[/TGT] п...


In [34]:
mixed_df = df.sample(frac=1).reset_index(drop=True)
mixed_df

,Full context,1 level,2 level,conn,marked_text
0,"Если бы не деды, мы бы не собирались сегодня с...",5,9,neutral,"если бы не [TGT]деды[/TGT], мы бы не собиралис..."
1,Кто начнет первый посылать деда? Я думаю Рамза...,6,14,neutral,кто начнет первый посылать [TGT]деда[/TGT]? я ...
2,"Но этих мужчин надо слушать, мне таких в жизни...",1,1,neutral,"но этих [TGT]мужчин[/TGT] надо слушать, мне та..."
3,", даже очень порядочные и вовсе не корыстные д...",7,15,positive,", даже очень порядочные и вовсе не корыстные [..."
4,"Я знаю, о чем говорю, так как был воспитан мат...",6,12,neutral,"я знаю, о чем говорю, так как был воспитан мат..."
...,...,...,...,...,...
4463,"мужиками, а не с теми безусыми юнцами , что см...",6,12,neutral,"мужиками, а не с теми безусыми [TGT]юнцами[/TG..."
4464,"Я бы даже, честно говоря, усомнилась, что чуви...",7,15,neutral,"я бы даже, честно говоря, усомнилась, что [TGT..."
4465,чел сверху мой ребенок,2,2,neutral,[TGT]чел[/TGT] сверху мой ребенок
4466,"Эй Бро, нас не испортить деньгами И чтобы не б...",3,3,positive,"эй [TGT]бро[/TGT], нас не испортить деньгами и..."


In [36]:
output_path = 'совсем окончательный датасет.xlsx'
mixed_df.to_excel(output_path, index=False)

# Подсчет количества токенов

In [3]:
df = pd.read_excel('/content/Совсем окончательный датасет.xlsx')
df.head()

,Full context,1 level,2 level,conn,marked_text
0,"Если бы не деды, мы бы не собирались сегодня с...",5,9,neutral,"если бы не [TGT]деды[/TGT], мы бы не собиралис..."
1,Кто начнет первый посылать деда? Я думаю Рамза...,6,14,neutral,кто начнет первый посылать [TGT]деда[/TGT]? я ...
2,"Но этих мужчин надо слушать, мне таких в жизни...",1,1,neutral,"но этих [TGT]мужчин[/TGT] надо слушать, мне та..."
3,", даже очень порядочные и вовсе не корыстные д...",7,15,positive,", даже очень порядочные и вовсе не корыстные [..."
4,"Я знаю, о чем говорю, так как был воспитан мат...",6,12,neutral,"я знаю, о чем говорю, так как был воспитан мат..."


In [5]:
!pip install nltk

In [6]:
import nltk

In [7]:
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

total_words = df['Full context'].apply(
    lambda x: len(word_tokenize(str(x))) if pd.notna(x) else 0
).sum()
total_words

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


np.int64(80650)

In [8]:
total_words = df['marked_text'].apply(
    lambda x: len(word_tokenize(str(x))) if pd.notna(x) else 0
).sum()
total_words

np.int64(95050)